In [1]:
import pandas as pd
import nltk
import re
from sklearn import *
import time

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\ProgramData\Anacon

In [2]:
def dataClean(tweets_raw):
	cleanTweets = []
	for each in tweets_raw:
		tweet = each
		tweet = tweet.lower() #convert to lowercase
		tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet) #remove URL
		tweet = re.sub(r'(\s)@\w+', r'', tweet) #remove usernames
		tweet = re.sub(r'@\w+', r'', tweet) #remove usernames
		tweet = re.sub('<[^<]+?>', '', tweet) #remove HTML tags
		tweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', tweet) #remove punctuation and special characters 
		lower_case = tweet.lower() #tokenization 
		words = lower_case.split()
		tweet = ' '.join([w for w in words if not w in nltk.corpus.stopwords.words("english")]) #remove stopwords
		ps = nltk.stem.PorterStemmer()
		stemmedTweet = [ps.stem(word) for word in tweet.split(" ")]
		stemmedTweet = " ".join(stemmedTweet)
		tweet = str(stemmedTweet)
		tweet = tweet.replace("'", "")
		tweet = tweet.replace("\"","")
		cleanTweets.append(tweet)
	return cleanTweets

In [3]:
def vectorization(clean_train_tweets):
	vec = feature_extraction.text.TfidfVectorizer(min_df = 0.00125, max_df = 0.7, sublinear_tf=True, use_idf=True, stop_words=u'english', analyzer= 'word', ngram_range=(1,5),lowercase=True)
	train_vectors = vec.fit_transform(clean_train_tweets)
	return train_vectors

In [4]:
#Loading the sheets into data frames

trainingFile = "train.xlsx"
df_obama = pd.read_excel(trainingFile,sheetname='Obama')
df_romney = pd.read_excel(trainingFile,sheetname='Romney')

#Removing the mixed class and the !!! class

df_obama = df_obama[(df_obama['Class'].isin((1,-1,0)))]
df_romney = df_romney[(df_romney['Class'].isin((1,-1,0)))]

#creating lists for raw tweets and classes

obama_tweets_raw = df_obama['Anootated tweet']
obama_class = df_obama['Class']
romney_tweets_raw = df_romney['Anootated tweet']
romney_class = df_romney['Class']

obama_tweets_raw = obama_tweets_raw.tolist()
romney_tweets_raw = romney_tweets_raw.tolist()
obama_class_train = obama_class.tolist()
romney_class_train = romney_class.tolist()

romney_tweets = dataClean(romney_tweets_raw) #romney tweets cleaning
obama_tweets = dataClean(obama_tweets_raw) #obama tweets cleaning

In [55]:
obama_tweets_vectors = obama_tweets#vectorization(obama_tweets)
romney_tweets_vectors = romney_tweets#vectorization(romney_tweets)
#obama_tweets_vectors[0][0]

In [6]:
'''
#obama
# Preparation for 10-fold cross validation
print(obama_tweets_vectors.shape)
n_samples = obama_tweets_vectors.shape[0]
n_fold = 10
fold_size = n_samples % n_fold
for fold in range(n_fold):
    train = []
    test = []
    if fold < n_fold - 1:
        for idx in range(n_samples):
            if idx < fold_size * fold or idx > fold_size * (fold+1):
                train.append(obama_tweets_vectors[idx])
        for idx in range(fold_size * fold, fold_size * (fold + 1)):
            test.append(obama_tweets_vectors[idx])
    else:
        for idx in range(fold_size * fold):
            train.append(obama_tweets_vectors[idx])
        for idx in range(fold_size * fold, n_samples):
            test.append(obama_tweets_vectors[idx])
len(test)
'''

'\n#obama\n# Preparation for 10-fold cross validation\nprint(obama_tweets_vectors.shape)\nn_samples = obama_tweets_vectors.shape[0]\nn_fold = 10\nfold_size = n_samples % n_fold\nfor fold in range(n_fold):\n    train = []\n    test = []\n    if fold < n_fold - 1:\n        for idx in range(n_samples):\n            if idx < fold_size * fold or idx > fold_size * (fold+1):\n                train.append(obama_tweets_vectors[idx])\n        for idx in range(fold_size * fold, fold_size * (fold + 1)):\n            test.append(obama_tweets_vectors[idx])\n    else:\n        for idx in range(fold_size * fold):\n            train.append(obama_tweets_vectors[idx])\n        for idx in range(fold_size * fold, n_samples):\n            test.append(obama_tweets_vectors[idx])\nlen(test)\n'

In [7]:
'''
import numpy as np

training = obama_tweets_vectors
print(training.shape)
obama_class_train = np.asarray(obama_class_train)
num_folds = 10
subset_size = training.shape[0]/num_folds
subset_size = round(subset_size,0)
print(subset_size)
for i in range(num_folds):
    print(i)
    testing_this_round = training[i*subset_size:][:subset_size]
    testing_this_round_class = obama_class_train[i*subset_size:][:subset_size]
    print(testing_this_round.shape)
    training_this_round = training[:i*subset_size] + training[(i+1)*subset_size:]
    training_this_round_class = obama_class_train[:i*subset_size] + obama_class_train[(i+1)*subset_size:]
    #print(len(training_this_round))
'''

'\nimport numpy as np\n\ntraining = obama_tweets_vectors\nprint(training.shape)\nobama_class_train = np.asarray(obama_class_train)\nnum_folds = 10\nsubset_size = training.shape[0]/num_folds\nsubset_size = round(subset_size,0)\nprint(subset_size)\nfor i in range(num_folds):\n    print(i)\n    testing_this_round = training[i*subset_size:][:subset_size]\n    testing_this_round_class = obama_class_train[i*subset_size:][:subset_size]\n    print(testing_this_round.shape)\n    training_this_round = training[:i*subset_size] + training[(i+1)*subset_size:]\n    training_this_round_class = obama_class_train[:i*subset_size] + obama_class_train[(i+1)*subset_size:]\n    #print(len(training_this_round))\n'

In [8]:

import numpy
from imblearn.over_sampling import SMOTE
from sklearn import svm

K = 10
for k in range(K):
    training = (x for i, x in enumerate(obama_tweets_vectors) if i % K != k)
    training = pd.DataFrame(training)
    training_class = (x for i, x in enumerate(obama_class_train) if i % K != k)
    training_class = pd.DataFrame(training_class)
    validation = [x for i, x in enumerate(obama_tweets_vectors) if i % K == k]
    validation_class = [x for i, x in enumerate(obama_class_train) if i % K == k]
    #print(training[0:5][0])
    #print(type(training),type(training_class))
    #print(len(validation),len(validation_class))
    sm = SMOTE(random_state=42)
    #training_class = numpy.asarray(training_class)
    #training = vectorization(training)
    #train, train_class = sm.fit_sample(training, training_class)
    #print(len(train),len(train_class))
    
    clf = svm.SVC(decision_function_shape='ovo')
    clf.fit(training, training_class)
    preds = clf.predict(validation)
    acc_score = metrics.accuracy_score(validation_class,preds)
    print(acc_score)
    

ValueError: setting an array element with a sequence.

In [39]:
from random import seed
from random import randrange

# Split a dataset into k folds
def cross_validation_data(dataset, folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(dataset.shape[0] / folds)
	for i in range(folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# test cross validation split
seed(1)
dataset = obama_tweets_vectors
dataset_folds = cross_validation_data(dataset, 10)

print(len(dataset_folds[1]))

AttributeError: 'list' object has no attribute 'shape'

In [40]:
from random import seed
from random import randrange

# Split a dataset into k folds
def cross_validation_label(dataset, folds=3):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / folds)
	for i in range(folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# test cross validation split
seed(1)
label = obama_class_train
label_folds = cross_validation_label(label,10)

print(len(label_folds[1]))


dataset = obama_tweets_vectors
dataset_folds = cross_validation_label(dataset, 10)

print(len(dataset_folds[1]))

547
547


In [60]:
#label_folds = label_folds.reshape(1, -1)
#dataset_folds = dataset_folds.reshape(1,-1)
#print(label_folds[i])
for i in range(0,10):
    #print(len(label_folds[i]),(len(dataset_folds[i])))
    #dataset_folds[i] = vectorization(dataset_folds[i])
    clf = linear_model.LogisticRegression(class_weight='balanced')
    #clf.fit(dataset_folds[i],label_folds[i])
    preds = model_selection.cross_val_predict(clf, obama_tweets_vectors, obama_class_train, cv=10)

AttributeError: lower not found